## MULTIWAY CUT LP relaxation
### Graph based on viewers rating movies
Here we solve the relaxed LP to find the multiway cut in a graph. The goal of the minimum multiway Cut is to find a set of edges $E_2 \subseteq E$ with minimum weight such that removing $E_2$ from $G$ separates all terminals nodes preselected.

In [ ]:
from algo import build_laplacian, solve_sdp, assignment_solution
from data_proc import build_a, build_graph, build_user_to_movies
import numpy as np
import networkx as nx

## Fake data to illustrate the integral solution
Easy multiway cut solution with 3 terminal nodes.

In [ ]:
W = np.array([[0,5,0,0,0,0,1,8],
              [5,0,1,0,0,0,0,0],
              [0,1,0,8,0,0,0,0],
              [0,0,8,0,1,0,0,0],
              [0,0,0,1,0,3,0,0],
              [0,0,0,0,3,0,3,0],
              [1,0,0,0,0,3,0,1],
              [8,0,0,0,0,0,1,0],
             ])
L = build_laplacian(W)
T = [0,2,5]
D = solve_multicut(L,T)
print(D)

In [ ]:
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 2
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W.copy()
W_gamma2[mask] = r[mask]
L = build_laplacian(W_gamma2)
D = solve_multicut(L,T)
print(D)
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

## Simulated data
Simulating a bipartite graph with viewers giving grades (between -2.5 and 2.5) to movies.

In [ ]:
G=nx.Graph()

G.add_edge(1,5,weight=1)
G.add_edge(1,6,weight=1)
G.add_edge(2,5,weight=1)
G.add_edge(2,6,weight=1)
G.add_edge(2,7,weight=-1)
G.add_edge(2,8,weight=-1)
G.add_edge(3,7,weight=1)
G.add_edge(3,8,weight=1)
G.add_edge(3,10,weight=2)
G.add_edge(4,7,weight=-2)
G.add_edge(4,9,weight=2)
G.add_edge(4,10,weight=1)
W = nx.to_numpy_matrix(G)
L = build_laplacian(W)
T = [5,7,9]
D = solve_multicut(L,T)
print(D)
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 2
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W.copy()
W_gamma2[mask] = r[mask]
L = build_laplacian(W_gamma2)
D = solve_multicut(L,T)
print(D)
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)

## Real data from themoviedb.org
Extracting  a sub graph of the entire database

In [ ]:
# building the summary dictionary
summary_dictionary = build_user_to_movies('movielens.tsv')
# unpacking of the dictionary
users_to_movies = summary_dictionary['users_to_movies']
n_users = summary_dictionary['n_users']
k_users = 6
n_movies = summary_dictionary['n_movies']
k_movies = 8
# building the rating matrix
a = build_a(n_users, k_users, n_movies, k_movies, users_to_movies)
# building the adjacency matrix
W = build_graph(k_users, k_movies, a)
print(W)

In [ ]:
# the laplacian of the graph
L = build_laplacian(W)
T = [0,2]
D = solve_multicut(L,T)
D_array=np.array(D)
print(np.round(D_array))
assignement_D = assignment_solution_lp(D_array, T, threshold=0.00001)
print(assignement_D)

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 2
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W.copy()
W_gamma2[mask] = r[mask]
L = build_laplacian(W_gamma2)
D = solve_multicut(L,T)
print(D)
assignement_D = assignment_solution_lp(D, T, threshold=0.00001)
print(assignement_D)